<a href="https://colab.research.google.com/github/75169/UMwfin/blob/Zadanie-3/Case%203.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install backtesting

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for backtesting: filename=Backtesting-0.3.3-py3-none-any.whl size=173916 sha256=1e68d7fb0e2c472dc19adfc0584f36df725aedd843a99d0ec713ffb47ee93259
  Stored in directory: /root/.cache/pip/wheels/e2/30/7f/19cbe31987c6ebdb47f1f510343249066711609e3da2d57176
Successfully built backtesting


In [2]:
!pip install yfinance

In [3]:
from backtesting import Backtest
from backtesting import Strategy
from backtesting.lib import crossover
from backtesting.test import SMA

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '
/usr/local/lib/python3.10/dist-packages/backtesting/test/__init__.py:8: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.read_csv(join(dirname(__file__), filename),
/usr/local/lib/python3.10/dist-packages/backtesting/test/__init__.py:8: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-cons

In [6]:
import datetime as dt
import yfinance as yf

MSFT = yf.download('MSFT', start='2024-01-01', end='2024-06-05')
MSFT.head()

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2024-01-02,373.859985,375.899994,366.769989,370.869995,369.518921,25258600
2024-01-03,369.010010,373.260010,368.510010,370.600006,369.249908,23083500
2024-01-04,370.670013,373.100006,367.170013,367.940002,366.599579,20901500
2024-01-05,368.970001,372.059998,366.500000,367.750000,366.410278,20987000
2024-01-08,369.299988,375.200012,369.010010,374.690002,373.325012,23134000


In [7]:
class SmaCross_withSL(Strategy):
    n1 = 10
    n2 = 25

    def init(self):
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)

    def next(self):
        if crossover(self.sma1, self.sma2):
          price = self.data.Close
          long_sl = price * 0.99
          long_tp = price * 1.05
          if self.position.is_short:
            self.position.close()
            self.buy(size=0.5, sl=long_sl, tp=long_tp)
          elif self.position.is_long:
            pass
          else:
            self.buy(size=0.5, sl=long_sl, tp=long_tp)

        elif crossover(self.sma2, self.sma1):
          price = self.data.Close
          short_sl = price * 1.01
          short_tp = price * 0.95
          if(self.position.is_long):
            self.position.close()
            self.sell(size = 0.5, sl=short_sl, tp=short_tp)
          elif self.position.is_short:
            pass
          else:
            self.sell(size = 0.5, sl=short_sl, tp=short_tp)

In [8]:
bt = Backtest(MSFT, SmaCross_withSL,
              cash=10000, commission=.002,
              exclusive_orders=True)

output = bt.run()
bt.plot()
print(output)

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


Start                     2024-01-02 00:00:00
End                       2024-06-04 00:00:00
Duration                    154 days 00:00:00
Exposure Time [%]                   38.317757
Equity Final [$]                  9680.257559
Equity Peak [$]                       10000.0
Return [%]                          -3.197424
Buy & Hold Return [%]               12.187562
Return (Ann.) [%]                   -7.367859
Volatility (Ann.) [%]                4.495426
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -3.197424
Avg. Drawdown [%]                   -3.197424
Max. Drawdown Duration       98 days 00:00:00
Avg. Drawdown Duration       98 days 00:00:00
# Trades                                    6
Win Rate [%]                              0.0
Best Trade [%]                      -0.554803
Worst Trade [%]                     -1.819001
Avg. Trade [%]                    

In [9]:
opt = bt.optimize(n1=[5,7,9,11,13,15], n2=[10,15, 20,25,30,35,40], maximize='Sharpe Ratio',
                  constraint=lambda p: p.n1 < p.n2)
print(opt)

print(opt._strategy)

Backtest.optimize:   0%|          | 0/2 [00:00<?, ?it/s]

Start                     2024-01-02 00:00:00
End                       2024-06-04 00:00:00
Duration                    154 days 00:00:00
Exposure Time [%]                   25.233645
Equity Final [$]                 10353.582469
Equity Peak [$]                  10571.089566
Return [%]                           3.535825
Buy & Hold Return [%]               12.187562
Return (Ann.) [%]                    8.527698
Volatility (Ann.) [%]                5.381224
Sharpe Ratio                         1.584713
Sortino Ratio                        2.728008
Calmar Ratio                         4.144557
Max. Drawdown [%]                   -2.057566
Avg. Drawdown [%]                   -0.660072
Max. Drawdown Duration       13 days 00:00:00
Avg. Drawdown Duration        6 days 00:00:00
# Trades                                    4
Win Rate [%]                             75.0
Best Trade [%]                       3.993474
Worst Trade [%]                     -1.514415
Avg. Trade [%]                    